In [0]:
import json
import openai
from typing import List, Dict, Any, Callable, Optional
from dataclasses import dataclass
from enum import Enum
import re
import os

class ToolType(Enum):
    CALCULATION = "calculation"
    DATABASE_QUERY = "database_query"
    API_CALL = "api_call"
    FILE_OPERATION = "file_operation"

@dataclass
class Tool:
    name: str
    description: str
    function: Callable
    parameters: Dict[str, Any]
    tool_type: ToolType

class LLMProvider:
    """Base class for LLM providers"""
    def generate_response(self, prompt: str) -> str:
        raise NotImplementedError

class OpenAIProvider(LLMProvider):
    """OpenAI GPT provider"""
    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
    
    def generate_response(self, prompt: str) -> str:
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=500,
                temperature=0.1
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"LLM Error: {str(e)}"

class LocalLLMProvider(LLMProvider):
    """Placeholder for local LLM (e.g., Ollama, Hugging Face)"""
    def __init__(self, model_name: str = "llama2"):
        self.model_name = model_name
    
    def generate_response(self, prompt: str) -> str:
        # This would integrate with local LLM APIs like Ollama
        # For demo purposes, using intelligent fallback
        return self._intelligent_fallback(prompt)
    
    def _intelligent_fallback(self, prompt: str) -> str:
        """Simulate intelligent response for demo purposes"""
        if "calculate" in prompt.lower():
            return json.dumps({
                "reasoning": "User is asking for a mathematical calculation",
                "tools_needed": ["calculator"],
                "execution_plan": "Extract the mathematical expression and use calculator tool"
            })
        elif any(keyword in prompt.lower() for keyword in ['sales', 'revenue', 'database', 'data']):
            return json.dumps({
                "reasoning": "User is requesting data from our database",
                "tools_needed": ["database_query"],
                "execution_plan": "Generate appropriate SQL query based on user request"
            })
        elif "weather" in prompt.lower():
            return json.dumps({
                "reasoning": "User wants weather information",
                "tools_needed": ["weather_api"],
                "execution_plan": "Extract location and call weather API"
            })
        else:
            return json.dumps({
                "reasoning": "General query that doesn't require specific tools",
                "tools_needed": [],
                "execution_plan": "Provide direct response"
            })

class IntelligentAgent:
    def __init__(self, name: str, role: str, llm_provider: LLMProvider):
        self.name = name
        self.role = role
        self.llm_provider = llm_provider
        self.tools = {}
        self.conversation_history = []
        self.working_memory = {}
    
    def register_tool(self, tool: Tool):
        """Register a new tool for the agent to use"""
        self.tools[tool.name] = tool
        print(f"🔧 Registered tool: {tool.name}")
    
    def available_tools_prompt(self) -> str:
        """Generate prompt describing available tools"""
        if not self.tools:
            return "No tools available."
        
        tools_desc = "Available tools:\n"
        for tool in self.tools.values():
            tools_desc += f"- {tool.name}: {tool.description} (Parameters: {tool.parameters})\n"
        return tools_desc
    
    def decide_and_act(self, user_input: str) -> str:
        """Main agent decision-making loop using LLM"""
        # Store user input
        self.conversation_history.append({"role": "user", "content": user_input})
        
        # Create decision prompt for LLM
        decision_prompt = f"""
        You are {self.name}, a {self.role}.
        
        {self.available_tools_prompt()}
        
        User Request: "{user_input}"
        
        Analyze the user's request and determine what tools (if any) need to be used.
        
        Respond with a JSON object containing:
        {{
            "reasoning": "brief explanation of your approach",
            "tools_needed": ["list", "of", "tool", "names"],
            "execution_plan": "step by step plan",
            "direct_response": "if no tools needed, provide direct answer here"
        }}
        
        Important: Only include tools that are actually available in the tools list above.
        """
        
        # Get LLM decision
        llm_response = self.llm_provider.generate_response(decision_prompt)
        
        try:
            # Parse LLM response
            decision = json.loads(llm_response)
            return self._execute_plan(user_input, decision)
        except json.JSONDecodeError:
            # Fallback if JSON parsing fails
            print("⚠️ LLM response was not valid JSON, using fallback logic")
            return self._fallback_execution(user_input)
    
    def _execute_plan(self, user_input: str, decision: Dict) -> str:
        """Execute the plan decided by the LLM"""
        response_parts = [f"🤔 Reasoning: {decision.get('reasoning', 'Processing request...')}\n"]
        
        tools_needed = decision.get('tools_needed', [])
        
        if not tools_needed:
            # No tools needed, return direct response
            direct_response = decision.get('direct_response', 'I understand your request.')
            return direct_response
        
        # Execute tools
        for tool_name in tools_needed:
            if tool_name in self.tools:
                tool = self.tools[tool_name]
                result = self._execute_tool(tool, user_input)
                response_parts.append(f"🔧 {tool_name}: {result}")
            else:
                response_parts.append(f"❌ Tool '{tool_name}' not available")
        
        return "\n".join(response_parts)
    
    def _execute_tool(self, tool: Tool, user_input: str) -> str:
        """Execute a specific tool with extracted parameters"""
        try:
            if tool.name == "calculator":
                # Extract mathematical expressions
                math_pattern = r'(\d+(?:\.\d+)?\s*[+\-*/]\s*\d+(?:\.\d+)?(?:\s*[+\-*/]\s*\d+(?:\.\d+)?)*)'
                matches = re.findall(math_pattern, user_input)
                if matches:
                    return tool.function(matches[0])
                else:
                    # Look for word problems
                    numbers = re.findall(r'\d+(?:\.\d+)?', user_input)
                    if len(numbers) >= 2:
                        if 'multiply' in user_input or '*' in user_input:
                            expr = f"{numbers[0]} * {numbers[1]}"
                        elif 'add' in user_input or '+' in user_input:
                            expr = f"{numbers[0]} + {numbers[1]}"
                        elif 'subtract' in user_input or '-' in user_input:
                            expr = f"{numbers[0]} - {numbers[1]}"
                        elif 'divide' in user_input or '/' in user_input:
                            expr = f"{numbers[0]} / {numbers[1]}"
                        else:
                            expr = f"{numbers[0]} + {numbers[1]}"  # default
                        return tool.function(expr)
            
            elif tool.name == "database_query":
                # Generate SQL based on user intent
                if 'sales' in user_input.lower() and ('q2' in user_input.lower() or '2024' in user_input.lower()):
                    query = "SELECT SUM(revenue) FROM sales WHERE quarter = 'Q2' AND year = 2024"
                elif 'customers' in user_input.lower():
                    query = "SELECT COUNT(*) FROM customers WHERE status = 'active'"
                elif 'satisfaction' in user_input.lower():
                    query = "SELECT AVG(satisfaction_score) FROM surveys WHERE date > '2024-01-01'"
                else:
                    query = "SELECT * FROM sales LIMIT 10"  # default query
                return tool.function(query)
            
            elif tool.name == "weather_api":
                # Extract location
                locations = ["toronto", "new york", "san francisco", "london", "paris"]
                location = "Toronto"  # default
                for loc in locations:
                    if loc in user_input.lower():
                        location = loc.title()
                        break
                return tool.function(location)
            
            else:
                return tool.function(user_input)
                
        except Exception as e:
            return f"Error executing {tool.name}: {str(e)}"
    
    def _fallback_execution(self, user_input: str) -> str:
        """Fallback execution when LLM parsing fails"""
        response_parts = []
        
        # Math-related queries
        if any(op in user_input for op in ['+', '-', '*', '/', 'calculate', 'compute']):
            if 'calculator' in self.tools:
                result = self._execute_tool(self.tools['calculator'], user_input)
                response_parts.append(f"🔧 Calculator: {result}")
        
        # Database queries
        if any(keyword in user_input.lower() for keyword in ['sales', 'revenue', 'customers', 'data']):
            if 'database_query' in self.tools:
                result = self._execute_tool(self.tools['database_query'], user_input)
                response_parts.append(f"🔧 Database: {result}")
        
        # Weather queries
        if 'weather' in user_input.lower():
            if 'weather_api' in self.tools:
                result = self._execute_tool(self.tools['weather_api'], user_input)
                response_parts.append(f"🔧 Weather: {result}")
        
        return "\n".join(response_parts) if response_parts else "I understand your request, but I need more specific information to help you."

# Tool implementations (same as before)
def calculator_tool(expression: str) -> str:
    """Safe calculator that evaluates mathematical expressions"""
    try:
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "Invalid expression"
    except Exception as e:
        return f"Error: {str(e)}"

def database_query_tool(sql_query: str) -> str:
    """Simulate database query execution"""
    simulated_results = {
        "SELECT SUM(revenue) FROM sales WHERE quarter = 'Q2' AND year = 2024": "$5.2M total revenue",
        "SELECT COUNT(*) FROM customers WHERE status = 'active'": "1,247 active customers",
        "SELECT AVG(satisfaction_score) FROM surveys WHERE date > '2024-01-01'": "4.7/5.0 average satisfaction",
        "SELECT * FROM sales LIMIT 10": "Top 10 recent sales records retrieved"
    }
    return simulated_results.get(sql_query, f"Executed: {sql_query}")

def weather_api_tool(location: str) -> str:
    """Simulate weather API call"""
    weather_data = {
        "Toronto": "22°C, Partly cloudy, 60% humidity",
        "New York": "18°C, Rainy, 80% humidity",
        "San Francisco": "16°C, Foggy, 85% humidity",
        "London": "15°C, Overcast, 75% humidity",
        "Paris": "20°C, Sunny, 55% humidity"
    }
    return weather_data.get(location, f"Weather data not available for {location}")

# Factory function to create agents with different LLM providers
def create_agent_with_llm(provider_type: str = "local", **kwargs):
    """Create an intelligent agent with specified LLM provider"""
    
    if provider_type == "openai":
        api_key = kwargs.get('api_key') or os.getenv('OPENAI_API_KEY')
        if not api_key:
            print("⚠️ No OpenAI API key provided, falling back to local LLM")
            llm_provider = LocalLLMProvider()
        else:
            model = kwargs.get('model', 'gpt-3.5-turbo')
            llm_provider = OpenAIProvider(api_key, model)
    else:
        # Default to local LLM
        model_name = kwargs.get('model_name', 'llama2')
        llm_provider = LocalLLMProvider(model_name)
    
    agent = IntelligentAgent("SalesBot", "Sales Data Analyst", llm_provider)
    
    # Register tools
    agent.register_tool(Tool(
        name="calculator",
        description="Performs mathematical calculations and arithmetic operations",
        function=calculator_tool,
        parameters={"expression": "string"},
        tool_type=ToolType.CALCULATION
    ))
    
    agent.register_tool(Tool(
        name="database_query",
        description="Queries the sales and customer database for business analytics",
        function=database_query_tool,
        parameters={"sql_query": "string"},
        tool_type=ToolType.DATABASE_QUERY
    ))
    
    agent.register_tool(Tool(
        name="weather_api",
        description="Gets current weather information for any location",
        function=weather_api_tool,
        parameters={"location": "string"},
        tool_type=ToolType.API_CALL
    ))
    
    return agent

# Demo function
def run_agent_demo():
    """Demonstrate the LLM-powered agent"""
    print(" LLM-Powered Intelligent Agent Demo")
    print("=" * 50)
    
    # Create agent (will use local LLM by default)
    # To use OpenAI, uncomment the line below and provide your API key
    # agent = create_agent_with_llm("openai", api_key="your-api-key-here")
    agent = create_agent_with_llm("local")
    
    test_scenarios = [
        "Calculate 25 + 75 - 30",
        "Show me our Q2 2024 sales revenue",
        "How many active customers do we have?",
        "What's the weather like in Toronto?",
        "What's the current weather in Paris?",
        "Can you tell me about inflation?"
    ]
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n Test {i}: {scenario}")
        print("-" * 40)
        response = agent.decide_and_act(scenario)
        print(f" Agent Response:\n{response}")
        print()

if __name__ == "__main__":
    run_agent_demo()

 LLM-Powered Intelligent Agent Demo
🔧 Registered tool: calculator
🔧 Registered tool: database_query
🔧 Registered tool: weather_api

 Test 1: Calculate 25 + 75 - 30
----------------------------------------
 Agent Response:
🤔 Reasoning: User is asking for a mathematical calculation

🔧 calculator: 25 + 75 - 30 = 70


 Test 2: Show me our Q2 2024 sales revenue
----------------------------------------
 Agent Response:
🤔 Reasoning: User is requesting data from our database

🔧 database_query: $5.2M total revenue


 Test 3: How many active customers do we have?
----------------------------------------
 Agent Response:
🤔 Reasoning: User is requesting data from our database

🔧 database_query: 1,247 active customers


 Test 4: What's the weather like in Toronto?
----------------------------------------
 Agent Response:
🤔 Reasoning: User is requesting data from our database

🔧 database_query: Top 10 recent sales records retrieved


 Test 5: What's the current weather in Paris?
--------------------